In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from vit_keras import vit

# Define the input shape
input_shape = (256, 256, 3)

# Define the VGG16-based CNN feature extractor
vgg16 = tf.keras.applications.VGG16(include_top=False, input_shape=input_shape)

# Define the ViT-based Transformer encoder
vit_model = vit.vit_b16(
    image_size=input_shape[0],
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
)

# Define the Decoder with U-Net architecture
def create_decoder_block(inputs, skip_features, output_filters):
    x = layers.UpSampling2D(size=(2, 2))(inputs)
    x = layers.Concatenate()([x, skip_features])
    x = layers.Conv2D(output_filters, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(output_filters, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

# Define the TransUNet model
def transunet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # CNN-based feature extraction
    cnn_features = vgg16(inputs)

    # Transformer-based global context extraction
    encoded_features = vit_model(cnn_features)

    # Decoder with U-Net architecture
    x = create_decoder_block(encoded_features, vgg16.layers[13].output, 256)
    x = create_decoder_block(x, vgg16.layers[9].output, 128)
    x = create_decoder_block(x, vgg16.layers[5].output, 64)
    x = create_decoder_block(x, vgg16.layers[2].output, 32)

    # Output layer
    outputs = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(x)

    # Define the model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Create the TransUNet model
model = transunet(input_shape)
